# Baseline Model

In [6]:
%load_ext autoreload
%autoreload 2

from utils import code
#from plot_libraries import setup_graphics

Parent directory /Users/nicholasrichers/Documents/GitHub/b2b_sales_project
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# load libraries and set plot parameters
import os, random, re, sys, time, warnings
import math
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

%matplotlib inline
sns.set()
pd.options.display.max_columns = None
setup_graphics()

In [7]:
import os, random, re, sys, time, warnings


#sys.path

from utils import code

In [16]:
import os 
  
# get current directory 
path = os.getcwd()  
# parent directory 
parent = os.path.dirname(path) 
print("Parent directory", parent) 

Current Directory /Users/nicholasrichers/Documents/GitHub/b2b_sales_project/notebooks

Parent directory /Users/nicholasrichers/Documents/GitHub/b2b_sales_project


### Analyzing Train DF

##### Features importantes
* Pct Organic Visits - Se esse número é alto, isso quer dizer que a empresa depende de SEO para gerar receita. Números extremos perto de 0% e perto de 100% provavelmente estão errados/não são confiáveis
* SEO Visits -  Quanto maior esse número, em geral, mais poderoso é o time de SEO, e mais faz sentido investir numa tecnologia como a nossa. Abaixo de 1M de visitas orgãnicas ou acima de 500M não faz muito sentido. (deveríamos descartar esses entao?)
* Page Count - Quanto maior esse número, em geral, mais poderoso é o time de SEO, e mais faz sentido investir numa tecnologia como a nossa. Abaixo de 25K não faz muito sentido.

In [5]:
df = pd.read_csv('data/trainDF.csv', delimiter=',', index_col=['Account_ID'])
df.shape

(455, 13)

In [7]:
df.tail()

,Number_Of_Opportunities,Number_of_Open_Opportunities,Activities_Last_30_Days,Employees,Pct_Organic_Visits,SEO_Visits,Annual_Revenue_converted,Adjusted_Industry,Account_ICP_Score,Page_Count,Alexa_Rank,Parent_Account_Status,Active_customer
Account_ID,,,,,,,,,,,,,
0012400000kkZyF,3.0,0.0,4.0,10000.0,0.10,10742515.0,1.000000e+09,Other,58.333,94900.0,260.0,-,0
001240000132Sos,5.0,1.0,17.0,3600.0,0.05,2473917.0,4.500000e+08,Other,58.333,484000.0,256.0,-,1
001240000132TKZ,7.0,3.0,17.0,3000.0,0.07,21176410.0,1.000000e+09,Other,50.000,47400.0,59.0,-,1
00124000015MpD7,6.0,1.0,0.0,22000.0,0.23,2239042.0,6.900000e+09,Travel,62.500,14912.0,2934.0,-,1
00124000018rlXA,4.0,1.0,0.0,37000.0,0.35,2933106.0,8.000000e+09,Classified,62.500,47880.0,5668.0,-,0


In [ ]:
#verificar o target

### Missing Values

In [ ]:
nan_mean = dataset.isna().mean()
nan_mean = nan_mean[nan_mean != 0].sort_values()
nan_mean

##### Fill NA Methods
* **Alexa_Rank:** MAX
* **Account_ICP_Score:** MEDIAN (industry)
* **Employees:** MEDIAN (industry)
* **Annual_Revenue_converted:** MEDIAN (industry)
* **Organic_Visits:** MIN
* **Page_Count:** MIN
* **Pct_Organic_Visits:**  MIN
* **Combined_Pages:** DROP

In [ ]:
fill_max = ['Alexa_Rank' ]
fill_min = ['Organic_Visits', 'Page_Count', 'Pct_Organic_Visits']
fill_median = ['Account_ICP_Score', 'Employees', 'Annual_Revenue_converted' ]
drop_missing_values = ['Combined_Pages']

In [ ]:
dataset[fill_max] = dataset[fill_max].fillna(dataset[fill_max].max())
dataset[fill_min] = dataset[fill_min].fillna(dataset[fill_min].min())
dataset.drop(columns=drop_missing_values, inplace=True)

In [ ]:
values_dict = dataset.groupby(['Adjusted_Industry'])[fill_median].median().to_dict()
for col in fill_median:
    dataset[col] = dataset[col].fillna(dataset['Adjusted_Industry'].map(values_dict[col]))